<a href="https://colab.research.google.com/github/sebavassou/CatsandDogsModelTransferLearning/blob/main/CatsandDogsModelTransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Visão Geral do Projeto
O objetivo é aplicar Transfer Learning em uma rede neural profunda usando Python no ambiente Google Colab. O projeto se baseará no exemplo do MNIST, mas será adaptado para classificar imagens de gatos e cachorros.
Dataset Cats vs Dogs
O dataset "Cats vs Dogs" é uma coleção extensa de imagens de gatos e cachorros, ideal para tarefas de classificação binária2. Algumas características importantes do dataset incluem:
•	Tamanho total: 1.04 GiB
•	Número de exemplos: 23,262 imagens no conjunto de treinamento
•	Formato das imagens: RGB (3 canais de cor)
•	Classes: 2 (gato e cachorro)
É importante notar que o dataset original continha algumas imagens corrompidas, mas essas foram removidas na versão atual2.
Implementação do Transfer Learning
Para implementar o Transfer Learning, você pode seguir estes passos:
1.	Carregar o dataset: Use a API do TensorFlow Datasets para carregar os dados:
python
import tensorflow_datasets as tfds

dataset, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)
2.	Pré-processar as imagens: Redimensione e normalize as imagens para se adequarem ao modelo pré-treinado que você escolher.
3.	Escolher um modelo base: Selecione um modelo pré-treinado, como VGG16, ResNet50 ou InceptionV3, disponível no Keras.
4.	Congelar as camadas base: Para preservar os pesos pré-treinados:
python
base_model.trainable = False
5.	Adicionar novas camadas: Adicione camadas densas no topo do modelo base para a classificação específica de gatos e cachorros.
6.	Compilar e treinar: Compile o modelo com um otimizador adequado e treine-o com os dados de gatos e cachorros.
7.	Fine-tuning: Opcionalmente, descongele algumas camadas superiores do modelo base e faça um fine-tuning com uma taxa de aprendizado baixa.
Considerações Adicionais
•	O dataset está disponível para download direto da Microsoft, com um tamanho de 786.7 MB3.
•	Utilize o Google Colab para aproveitar os recursos de GPU gratuitos, o que pode acelerar significativamente o treinamento do modelo.
•	Considere usar técnicas de aumento de dados (data augmentation) para melhorar a generalização do modelo, especialmente considerando o número relativamente pequeno de imagens no dataset.


In [ ]:
# Importando as bibliotecas necessárias
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

# Configurando o TensorFlow para usar GPU, se disponível
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Definindo parâmetros
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

# Baixando e extraindo o dataset
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
!unzip -q kagglecatsanddogs_5340.zip

# Preparando os geradores de dados
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    'PetImages',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'PetImages',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

# Carregando o modelo base VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Adicionando novas camadas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compilando o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinando o modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

# Plotando os resultados
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia do Modelo')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda do Modelo')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()

plt.tight_layout()
plt.show()

# Fine-tuning (opcional)
base_model.trainable = True
for layer in base_model.layers[:15]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=5
)

# Salvando o modelo
model.save('cats_vs_dogs_model.h5')


--2025-01-27 18:40:25--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.196.50.17, 2600:1407:7400:1187::317f, 2600:1407:7400:1184::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.196.50.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip’

kagglecatsanddogs_5 100%[===================>] 786.67M   108MB/s    in 8.6s    

2025-01-27 18:40:34 (91.3 MB/s) - ‘kagglecatsanddogs_5340.zip’ saved [824887076/824887076]

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 76/625 ━━━━━━━━━━━━━━━━━━━━ 3:08:09 21s/step - accuracy: 0.6431 - loss: 0.6150

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x792c69963330>
Traceback (most recent call last):

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py", line 265, in _get_iterator
    for i, batch in enumerate(gen_

In [ ]:
# Importando as bibliotecas necessárias
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import os
from PIL import UnidentifiedImageError


# Configurando o TensorFlow para usar GPU, se disponível
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Definindo parâmetros
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

# Baixando e extraindo o dataset
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
!unzip -q kagglecatsanddogs_5340.zip

# Preparando os geradores de dados
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    'PetImages',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'PetImages',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

# Carregando o modelo base VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Adicionando novas camadas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compilando o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])y


# Function to handle image loading errors
def load_image_with_error_handling(img_path):
    try:
        img = tf.keras.utils.load_img(img_path, target_size=IMG_SIZE)
        img = tf.keras.utils.img_to_array(img)
        return img
    except (UnidentifiedImageError, OSError) as e:
        print(f"Error loading image {img_path}: {e}")
        return None


# Custom generator with error handling
def data_generator(generator):
    while True:
        batch_x, batch_y = next(generator)
        # Filter out images that couldn't be loaded
        valid_indices = [i for i, img in enumerate(batch_x) if img is not None]
        if valid_indices:
            yield batch_x[valid_indices], batch_y[valid_indices]
        else:
            # If all images in the batch are invalid, skip the batch
            continue


# Create custom generators for training and validation
train_generator_wrapped = data_generator(train_generator)
validation_generator_wrapped = data_generator(validation_generator)

# Treinando o modelo
# Now use the wrapped generators
history = model.fit(
    train_generator_wrapped,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator_wrapped,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)
# Plotando os resultados
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia do Modelo')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda do Modelo')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()

plt.tight_layout()
plt.show()

# Fine-tuning (opcional)
base_model.trainable = True
for layer in base_model.layers[:15]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=5
)

# Salvando o modelo
model.save('cats_vs_dogs_model.h5')


--2025-01-27 19:34:37--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.62.142.15, 2600:1407:7400:1187::317f, 2600:1407:7400:1184::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.62.142.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip.1’

kagglecatsanddogs_5 100%[===================>] 786.67M   112MB/s    in 7.0s    

2025-01-27 19:34:44 (112 MB/s) - ‘kagglecatsanddogs_5340.zip.1’ saved [824887076/824887076]

replace PetImages/Cat/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace PetImages/Cat/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
A
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/10
103/625 ━━━━━━━━━━━━━━━━━━━━ 3:03:33 21s/step - accuracy: 0.6956 - loss: 0.5707

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


567/625 ━━━━━━━━━━━━━━━━━━━━ 19:55 21s/step - accuracy: 0.8018 - loss: 0.4209

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x792c5bc1df30>
Traceback (most recent call last):

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/generator_data_adapter.py", line 52, in get_tf_iterator
    for batch in self.generat

In [1]:
# Importando as bibliotecas necessárias
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import os
from PIL import UnidentifiedImageError

In [2]:
# Configurando o TensorFlow para usar GPU, se disponível
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
# Definindo parâmetros
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

In [4]:
# Baixando e extraindo o dataset
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
!unzip -q kagglecatsanddogs_5340.zip


--2025-01-28 15:04:35--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.216.93.77, 2600:1408:20:484::317f, 2600:1408:20:483::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.216.93.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip’

kagglecatsanddogs_5 100%[===================>] 786.67M   108MB/s    in 8.8s    

2025-01-28 15:04:44 (89.5 MB/s) - ‘kagglecatsanddogs_5340.zip’ saved [824887076/824887076]



In [5]:
# Preparando os geradores de dados
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    'PetImages',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'PetImages',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [11]:
# Carregando o modelo base VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False


In [12]:
# Adicionando novas camadas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

In [14]:
# Compilando o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
# Function to handle image loading errors
def load_image_with_error_handling(img_path):
    try:
        img = tf.keras.utils.load_img(img_path, target_size=IMG_SIZE)
        img = tf.keras.utils.img_to_array(img)
        return img
    except (UnidentifiedImageError, OSError) as e:
        print(f"Error loading image {img_path}: {e}")
        return None

In [17]:
# Custom generator with error handling
def data_generator(generator):
    while True:
        batch_x, batch_y = next(generator)
        # Filter out images that couldn't be loaded
        valid_indices = [i for i, img in enumerate(batch_x) if img is not None]
        if valid_indices:
            yield batch_x[valid_indices], batch_y[valid_indices]
        else:
            # If all images in the batch are invalid, skip the batch
            continue

In [18]:
# Create custom generators for training and validation
train_generator_wrapped = data_generator(train_generator)
validation_generator_wrapped = data_generator(validation_generator)


In [ ]:
# Treinando o modelo
# Now use the wrapped generators
history = model.fit(
    train_generator_wrapped,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator_wrapped,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

In [ ]:
# Plotando os resultados
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia do Modelo')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda do Modelo')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Fine-tuning (opcional)
base_model.trainable = True
for layer in base_model.layers[:15]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=5
)


In [ ]:
# Salvando o modelo
model.save('cats_vs_dogs_model.h5')
